# Segmenting and Clustering Neighborhoods in Toronto

## Assignment Instructions:

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

Your submission will be a link to your Jupyter Notebook on your Github repository.

## Web Scraping:

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

rawdata = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(rawdata,'lxml')

data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    if (index == 0):
        columns = section
    else:
        data.append(section)

## Dataframe Creation:

In [2]:

#convert list into Pandas DataFrame
toronto_df = pd.DataFrame(data = data,columns = columns)

toronto_df.head()


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Dataframe Formatting:

Remove Boroughs that are 'Not assigned'

In [3]:
toronto_df = toronto_df[toronto_df['Borough'] != 'Not assigned']
toronto_df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"



For neighbourhoods that are "Not assigned", apply their Borough as their Neighbourhood

In [4]:
toronto_df['Neighbourhood'].replace("Not assigned", toronto_df["Borough"],inplace=True)
toronto_df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"



Group all neighborhoods with the same postal code

In [5]:
toronto_df = toronto_df.groupby(["Postal Code", "Borough"])["Neighbourhood"].apply(", ".join).reset_index()
toronto_df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Final Dataframe Shape:

In [6]:
toronto_df.head()

print("Shape:", toronto_df.shape)

Shape: (103, 3)


---

## Retrieving Longitude and Latitude Coordinates:

In [7]:
geo_df = pd.read_csv("https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv")
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge Neigbourhood and Latitude/Logitude Data

In [8]:
TorGeo_df = pd.merge(toronto_df, geo_df, how='left', left_on = 'Postal Code', right_on = 'Postal Code')
TorGeo_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


---

## Exploring the Toronto Data:

Installing Folium

In [9]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


Importing the necessary libraries

In [10]:
import folium
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize 

Obtaining the correct latitude and longitude coordinates using Geopy

In [11]:
geolocator = Nominatim(user_agent="where_is_toronto")
location = geolocator.geocode('Toronto, Ontario')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto, Ontario are 43.6534817, -79.3839347.


Displaying the map of Toronto, Ontario with Postal Code Markers (Purple) using folium

In [12]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [13]:
for lat, long, borough, neighborhood in zip(
        TorGeo_df['Latitude'], 
        TorGeo_df['Longitude'], 
        TorGeo_df['Borough'], 
        TorGeo_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=7,
        popup=label,
        color='purple',
        fill=True,
        fill_color='mediumpurple',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

Determine what the unique Boroughs of Toronto are

In [14]:
TorGeo_df['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke'], dtype=object)

Creating a new Dataframe that only contain Boroughs that have "Toronto" in their names

In [15]:
TorBor_df = TorGeo_df[TorGeo_df['Borough'].str.contains("Toronto")].reset_index(drop=True)
TorBor_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Shape of the new Dataframe

In [16]:
TorBor_df.shape

(39, 5)

Displaying the map of Toronto Boroughs with Postal Code Markers (Purple) using folium

In [17]:
map_TorBor = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, long, borough, neighborhood in zip(
        TorBor_df['Latitude'], 
        TorBor_df['Longitude'], 
        TorBor_df['Borough'], 
        TorBor_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=7,
        popup=label,
        color='purple',
        fill=True,
        fill_color='mediumpurple',
        fill_opacity=0.7,
        parse_html=False).add_to(map_TorBor)  

map_TorBor

Foursquare Credentials

In [18]:
CLIENT_ID = 'J2ZYJERFLAGHCRQHEJUFYWI2XA5IA34UZ3VJ4A3152D0KZIK'
CLIENT_SECRET = 'EXF5OUXT4ECIDTN45HT1RJ3HKAEGZACL3OM0DEYZ2SBFDP33'
VERSION = '20200311'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: J2ZYJERFLAGHCRQHEJUFYWI2XA5IA34UZ3VJ4A3152D0KZIK
CLIENT_SECRET:EXF5OUXT4ECIDTN45HT1RJ3HKAEGZACL3OM0DEYZ2SBFDP33


---

# Exploring the Toronto Boroughs

Setup of the Foursquare API Query (50 Venues; 500 meter Radius) and a Function to retrieve Venue information

In [19]:
LIMIT = 50
radius = 500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
            
# create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
# make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
# Creating a function to extract Venue information from Foursquare API json
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Creating a Dataframe to store the Venue information about each Neighbourhood

In [20]:
TorBor_venues = getNearbyVenues(names=TorBor_df['Neighbourhood'],
                                   latitudes=TorBor_df['Latitude'],
                                   longitudes=TorBor_df['Longitude']
                                  )


In [21]:
TorBor_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


Listing the number of venues within each Neigbourhood

In [22]:
TorBor_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,50,50,50,50,50,50
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,50,50,50,50,50,50
Christie,16,16,16,16,16,16
Church and Wellesley,50,50,50,50,50,50
"Commerce Court, Victoria Hotel",50,50,50,50,50,50
Davisville,33,33,33,33,33,33


In [23]:
print('There are {} uniques categories.'.format(len(TorBor_venues['Venue Category'].unique())))

There are 214 uniques categories.


There are a wide variety of venues (n = 214) that are well dispersed within the different neighbourhoods. This raw dataset should be adequate for further analysis!

---

# Analyzing each Toronto Neighbourhood

The data we have is nominal, but not ordinal, therefore we will use One-Hot encoding to convert the categorical data to numerical data to perform further analyses. 

In [24]:
# one hot encoding
TorBor_onehot = pd.get_dummies(TorBor_venues[['Venue Category']], prefix="", prefix_sep="")
TorBor_onehot['Neighbourhood'] = TorBor_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [TorBor_onehot.columns[-1]] + list(TorBor_onehot.columns[:-1])
TorBor_onehot = TorBor_onehot[fixed_columns]

TorBor_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Grouping rows by neighbourhood and determing the mean of the frequency of occurrence for each category


In [25]:
TorBor_grouped = TorBor_onehot.groupby('Neighbourhood').mean().reset_index()
TorBor_grouped.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.00,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00
2,"Business reply mail Processing Centre, South C...",0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0625,0.0625,0.0625,0.125,0.125,0.0625,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00
4,Central Bay Street,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.02,0.02


Determing the Top 5 most common venues for each neighbourhood

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = TorBor_grouped['Neighbourhood']

for ind in np.arange(TorBor_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(TorBor_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Coffee Shop,Farmers Market,Beer Bar,Cocktail Bar,Seafood Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Nightclub,Coffee Shop,Breakfast Spot,Bakery
2,"Business reply mail Processing Centre, South C...",Pizza Place,Gym / Fitness Center,Recording Studio,Restaurant,Light Rail Station
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport,Bar,Harbor / Marina
4,Central Bay Street,Coffee Shop,Bubble Tea Shop,Café,Italian Restaurant,Burger Joint


---

# Clustering the Neighbourhood data by Venue

Using the k-means algorithm from scikitlearn to cluster the neighbourhood data into 5 clusters.



In [27]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

# set number of clusters
kclusters = 5

TorBor_grouped_clustering = TorBor_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(TorBor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

Creating a new dataframe that includes each cluster and the Top 5 most common venues for each Neighbourhood.


In [28]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

TorBor_merged = TorBor_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
TorBor_merged = TorBor_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

TorBor_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Trail,Neighborhood,Pub,Health Food Store,Yoga Studio
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,2,Park,Fish & Chips Shop,Sandwich Place,Brewery,Liquor Store
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Coffee Shop,Bakery,Gastropub,American Restaurant,Brewery
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Bus Line,Swim School,Electronics Store,Dumpling Restaurant


Visualizing the Cluster Results using Folium

In [29]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        TorBor_merged['Latitude'], 
        TorBor_merged['Longitude'], 
        TorBor_merged['Neighbourhood'], 
        TorBor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The five cluster are identified within the map above, Turquoise, Blue, Red, Purple and Orange markers signify the different clusters. 

---

# Examining the different Toronto Neighbourhood clusters

The first cluster has grouped the neighbourhoods with Parks as their most common venue (Red Markers)

In [30]:
TorBor_merged.loc[TorBor_merged['Cluster Labels'] == 0, TorBor_merged.columns[[1] + list(range(5, TorBor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,Central Toronto,0,Park,Bus Line,Swim School,Electronics Store,Dumpling Restaurant
10,Downtown Toronto,0,Park,Playground,Trail,Cuban Restaurant,Donut Shop
23,Central Toronto,0,Jewelry Store,Park,Trail,Sushi Restaurant,Dance Studio


The second cluster has grouped the neighbourhoods with Playgrounds as their most common venue (Purple Marker)

In [31]:
TorBor_merged.loc[TorBor_merged['Cluster Labels'] == 1, TorBor_merged.columns[[1] + list(range(5, TorBor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,Central Toronto,1,Playground,Trail,Yoga Studio,Cupcake Shop,Dumpling Restaurant


The third cluster has grouped the neighbourhoods with Coffee Shops as their most common venue (Blue Markers)

In [32]:
TorBor_merged.loc[TorBor_merged['Cluster Labels'] == 2, TorBor_merged.columns[[1] + list(range(5, TorBor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,East Toronto,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop
2,East Toronto,2,Park,Fish & Chips Shop,Sandwich Place,Brewery,Liquor Store
3,East Toronto,2,Coffee Shop,Bakery,Gastropub,American Restaurant,Brewery
5,Central Toronto,2,Dance Studio,Hotel,Park,Breakfast Spot,Dog Run
6,Central Toronto,2,Coffee Shop,Clothing Store,Bagel Shop,Furniture / Home Store,Ice Cream Shop
7,Central Toronto,2,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Sushi Restaurant
9,Central Toronto,2,Coffee Shop,Sushi Restaurant,American Restaurant,Bagel Shop,Supermarket
11,Downtown Toronto,2,Coffee Shop,Restaurant,Café,Pizza Place,Market
12,Downtown Toronto,2,Sushi Restaurant,Coffee Shop,Gay Bar,Yoga Studio,Men's Store
13,Downtown Toronto,2,Coffee Shop,Park,Bakery,Pub,Breakfast Spot


The fourth cluster has grouped the neighbourhoods with Neighbourhoods as their most common venue (Turquoise Marker)

In [33]:
TorBor_merged.loc[TorBor_merged['Cluster Labels'] == 3, TorBor_merged.columns[[1] + list(range(5, TorBor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,East Toronto,3,Trail,Neighborhood,Pub,Health Food Store,Yoga Studio


The fifth cluster has grouped the neighbourhoods with Music Venues as their most common venue (Orange Marker)

In [34]:
TorBor_merged.loc[TorBor_merged['Cluster Labels'] == 4, TorBor_merged.columns[[1] + list(range(5, TorBor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
22,Central Toronto,4,Garden,Music Venue,Yoga Studio,Dance Studio,Dumpling Restaurant
